In [0]:
# ================================================================
#  Validation Notebook Cell — Verify TMDB + Discogs Extract Outputs
# ================================================================

from pyspark.sql import SparkSession

# 1️⃣ Configure Spark with ADLS credentials from markscope
from pyspark.dbutils import DBUtils
dbutils = DBUtils(spark)

STORAGE_ACCOUNT = dbutils.secrets.get("markscope", "azure-storage-account-name").strip()
STORAGE_KEY = dbutils.secrets.get("markscope", "azure-storage-account-key").strip()
BASE_URI = f"abfss://raw@{STORAGE_ACCOUNT}.dfs.core.windows.net"

spark = (
    SparkSession.builder.appName("Validate_Extract_Outputs")
    .config(f"fs.azure.account.key.{STORAGE_ACCOUNT}.dfs.core.windows.net", STORAGE_KEY)
    .getOrCreate()
)

# 2️⃣ Define paths
tmdb_path = f"{BASE_URI}/raw/tmdb/"
discogs_path = f"{BASE_URI}/raw/discogs/"

# 3️⃣ Load Parquet outputs
tmdb_df = spark.read.parquet(tmdb_path)
discogs_df = spark.read.parquet(discogs_path)

# 4️⃣ Display summary info
print("✅ TMDB Extract")
tmdb_df.printSchema()
print(f"TMDB record count: {tmdb_df.count()}")
display(tmdb_df.limit(10))

print("✅ Discogs Extract")
discogs_df.printSchema()
print(f"Discogs record count: {discogs_df.count()}")
display(discogs_df.limit(10))

# 5️⃣ Sanity check: sample join keys overlap
tmdb_years = [r["tmdb_year"] for r in tmdb_df.select("tmdb_year").distinct().collect()]
discogs_years = [r["discogs_year"] for r in discogs_df.select("discogs_year").distinct().collect()]
overlap = set(tmdb_years) & set(discogs_years)
print(f"🎯 Overlapping year values: {sorted(list(overlap))[:10]}")

print("🧾 Validation complete — both extract outputs verified.")


# ================================================================
#  Validation Extension — Verify TMDB→Discogs Candidate Output
# ================================================================

print("\n✅ Candidate Generation Output — Step 03 (Pandas-based)")

# 6️⃣ Define candidate path (from Step03PrepareTMDBDiscogsCandidates)
candidates_path = f"{BASE_URI}/raw/intermediate/tmdb_discogs_candidates/"

# 7️⃣ Read the resulting Parquet from ADLS
try:
    candidates_df = spark.read.parquet(candidates_path)
    print("✅ Candidate Parquet loaded successfully.\n")
    candidates_df.printSchema()
    print(f"Candidate record count: {candidates_df.count()}")
    display(candidates_df.limit(10))
except Exception as e:
    print(f"❌ Failed to load candidates Parquet: {e}")

# 8️⃣ Quick sanity checks
if "tmdb_title_norm" in candidates_df.columns and "discogs_title_norm" in candidates_df.columns:
    overlap_titles = (
        candidates_df.select("tmdb_title_norm")
        .intersect(candidates_df.select("discogs_title_norm"))
        .count()
    )
    print(f"🔍 Normalized title overlaps (identical values): {overlap_titles}")
else:
    print("⚠️ Normalized title columns not present; skipping overlap check.")

print("🧭 Validation complete — TMDB + Discogs + Candidate outputs verified.")




In [0]:
from scripts.inventory_pipeline_outputs import run_inventory
run_inventory()


In [0]:
display(dbutils.fs.mounts())

In [0]:
# Replace with your actual values
storage_account = "ungcapstor01"
container_name = "raw"

# Retrieve the key from your Databricks secret scope
account_key = dbutils.secrets.get("markscope", "azure-storage-account-key")

# Unmount first (safe to ignore errors if not mounted)
try:
    dbutils.fs.unmount("/mnt/raw")
except:
    pass

# ✅ Correct scheme: abfss (NOT abfs)
dbutils.fs.mount(
    source = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/",
    mount_point = "/mnt/raw",
    extra_configs = {f"fs.azure.account.key.{storage_account}.dfs.core.windows.net": account_key}
)

display(dbutils.fs.mounts())
